## Access Rights/Restricted

**Observations**  
-Identification de données à accès restreint.

**Traitements**  
-Identification des données restreintes à partir des labels de la métadonnée *access_rights*, comparaison avec la métadonnée dérivée *restricted* (booléenne).

In [ ]:
import pandas as pd
import json

from utils.utils import wrapper_engine, get_value_from_json_serialization
engine = wrapper_engine('config.ini')

from utils import custom_settings
from itables import show
custom_settings.apply_itable_custom_settings()

In [ ]:
# Load data
with engine.connect() as connection:
    package = pd.read_sql_table(
        table_name="package", 
        con=connection,
        schema="public",
        )

    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )

**Nombre de datasets comportant la métadonnée access_rights**

In [ ]:
access_rights = package_extra[package_extra["key"]=="access_rights"].copy(deep=True)
access_rights["access_rights_label"] = access_rights["value"].apply(lambda x: get_value_from_json_serialization(x, "label")).apply(lambda x: [json.loads(val)["fr"] for val in x] )
show(access_rights[["package_id", "access_rights_label"]].rename(columns={"package_id":"datasets" }).describe())

In [ ]:
## Explode access_rights_label currently resolved as a list of labels, to display a single label per row.
access_rights = access_rights.explode('access_rights_label').reset_index(drop=True)

# Patterns identified manually from the previously exported access rights labels.
pattern_restricted_access = [
    "Accès possible",
    "la communication de ces données reste réservée aux seuls services",
    "Donnée à diffusion restreinte",
    "la donnée ne doit donc pas être en libre accès",
    "Donnée contenant des informations personnelles à diffusion restreinte"
]

access_rights["restricted"] = access_rights["access_rights_label"].apply(lambda x: any(label in x for label in pattern_restricted_access))
columns = ["title", "notes","access_rights_label"]
restricted_datasets = package.merge(access_rights[access_rights.restricted], how="inner", left_on="id", right_on="package_id")[columns]
show(restricted_datasets)

Métadonnée *restricted_access*, dérivée des labels *access_rights*

In [ ]:
restricted = package_extra[package_extra["key"]=="restricted_access"].copy(deep=True)
show(restricted[["package_id", "value"]].rename(columns={"package_id":"datasets", "value":"restricted_access"}).describe())